# Econ 8210 Quant Macro, Homework 1
## Part 3 - Dynamic Programming
Haosi Shen, Fall 2024

In [2]:
# Housekeeping
import numpy as np
import pandas as pd
import math
import time
import torch

np.random.seed(42) 

Consider the following model. There is a representative household with preferences over private consumption $c_t$, government consumption $g_t$, and labor $l_t$:

\begin{equation*}
\mathbb{E}_0 \sum_{t=0}^{\infty} 0.97^t \Big( \log{c_t}+0.2\log{g_t} -\frac{l_t^2}{2} \Big)
\end{equation*}

The household consumes, saves, works, and pays labor taxes, with a budget constraint:

\begin{equation*}
c_t + i_t = (1-\tau_t)w_t l_t + r_t k_t
\end{equation*}

where the tax rate follows a 3-state Markov chain that takes values in:
\begin{equation*}
\tau_t \in \{0.20, 0.25, 0.30\}
\end{equation*}
\end{pmatrix}

There is a production function:
\begin{equation*}
c_t + i_t + g_t = e^{z_t} k_{t}^{0.33} l_{t}^{0.67}
\end{equation*}

with a law of motion for capital with investment adjustment costs:
\begin{equation*}
k_{t+1} = 0.9 k_t + \Big(1-0.05 \big(\frac{i_t}{i_{t-1}}-1\big)^2 \Big) i_t
\end{equation*}

and a technology level $z_t$ that follows a 5-state Markov chain that takes values in:
\begin{equation*}
z_t \in \{−0.0673, −0.0336, 0, 0.0336, 0.0673\}
\end{equation*}

Finally, there is a government that uses taxes to pay for government consumption with a
balanced budget period by period:
\begin{equation*}
g_t = \tau_t w_t l_t
\end{equation*}

In [3]:
# ===================== DEFINE PARAMETERS =====================
beta = 0.97  # time discount factor
alpha = 0.33  # capital share
delta = 0.1  # capital depreciation rate
theta_c = 0.2  # weight on govt consumption in util
k_ss = 1.0  # steady-state capital (init guess)
i_ss = 0.1  # steady-state investment (init guess)

# Tax rates and transition matrix
tau_values = np.array([0.2, 0.25, 0.3])
tau_trans = np.array([[0.9, 0.1, 0.0],
                      [0.05, 0.9, 0.05],
                      [0.0, 0.1, 0.9]])

# Technology levels and transition matrix
z_values = np.array([-0.0673, -0.0336, 0, 0.0336, 0.0673])
z_trans = np.array([[0.9727, 0.0273, 0, 0, 0],
                    [0.0041, 0.9806, 0.0153, 0, 0],
                    [0, 0.0082, 0.9836, 0.0082, 0],
                    [0, 0, 0.0153, 0.9806, 0.0041],
                    [0, 0, 0, 0.0273, 0.9727]])
